# 26、K-Means（K-均值算法）：如何给20支亚洲球队做聚类

K-Means是一种非监督的学习，解决的是聚类的问题。K代表的是K类，Means带班表的是中心，你可以理解这个算法的本质是确定K类的中心点，当找到了这些中心点，也就完成了聚类。

思考三个问题：
- 1、如何确定K类的中心点？
- 2、如何将其他点划分到K类中？
- 3、如何区分K-Means与KNN？

## K-Means的工作原理

1、可以经验划分每个类的中心点

2、确定中心点之后，每个赝本会被划分到不同的中心点的类当中

3、K中心点是可以变化的，每一次遍历计算距离之后，都会重新计算出中心点，当中心点的位置不再改变之后就可以说是确定了。

4、我们只需要给定有个初始的K值，一般算法会随机设置初始的中心点。

K—Means的工作原理：
- 1、选取K个点作为初始的类中心点，这些点一般都是从数据集中随机抽取的；
- 2、将每个点分配到最近的类的中心点，这样就形成了K个类，然后重新计算每个类的中心点；
- 3、重复第二步，直到类不再发生变化，或者也可以设置最大的迭代次数，这样即使类中心点发生变化，但是只要到达迭代的次数就会结束。

## 如何给亚洲球队做聚类

1、有如下的数据：2015-2019年亚洲球队的排名：
![](亚洲球队排名（原始）.png)
2、对数据进行规范化，将数值划分到[0,1]或者按照均值为0，方差为1的正态分布进行规范化，这里采用的是Min-max数据规范化到[0,1]:
![](亚洲球队数据规范化.png)
3、随机选择中国、日本、韩国为三个类的中心点，计算这些球队到中心点的距离，计算距离的方法：欧氏、曼哈顿、切比雪夫、余弦。这里采用的是欧氏距离。得到的计算结果如下：
![](球队到中心点的距离.png)
4、重新计算出三个中心点，使用取平均值的方法，然后根据新的中心点按照距离的远近重新划分球队的分类，之后再根据球队的分类更新中心点位置，一直迭代下去，直到分类分类不再发生变化为止，结果如下：
![](球队迭代结果.png)

最后我们可以看出第一梯队的有日本、韩国、伊朗、沙特、澳洲；第二梯队的有中国、伊拉克、阿联酋、乌兹别克斯坦；第三梯队的有卡塔尔、泰国、越南、阿曼、巴林、朝鲜、印尼、叙利亚、约旦、科威特和巴勒斯坦。

## 如何使用sklearn中的K-Means算法

sklearn是Python的机器学习工具库，如果能够从功能上来划分，sklearn可以实现分类、聚类、回归、降维、模型选择和预处理等功能。这里我们使用的是sklearn的聚类函数库，因此需要引用工具包，具体的代码如下：

In [ ]:
from sklearn.cluster import KMeans

当然K-Means知识sklearn.cluster中的一个聚类库，实际上包括K-Means在内，sklearn.cluster 一共提供了9中聚类的方法，比如Mean-shift，DNSCAN，Spectral clustering（谱聚类）等。这些聚类方法的原理和K-Means不同。

K-Means的创建方式：

In [ ]:
KMeans(n_clusters=8, init='k-means++', n_init=10, max_iter=300, 
       tol=0.0001, precompute_distances='auto', verbose=0, 
       random_state=None, copy_x=True, n_jobs=1,algorithm='auto')

在K-Means类创建的过程中，有一些主要的参数：

- n_clusters:即K值，一般盱眙啊多试一些K值来保证更好的聚类效果。你可以随机设置一些K值，然后选择聚类效果好的作为最终的K值；
- max_iter：最大的迭代次数，如果聚类很难收敛的话，设置最大的迭代次数可以让我们及时得到反馈结果，否则程序运行时间会非常的长；
- n_init：初始化中心点的运算次数，默认是10。程序是否能够快速收敛和中心点的选择关系非常大，所以在中心点选择上会多花一些时间，来争取整体的时间上的快速收敛还是非常值得的。由于每一次中心点都是随机生成的，这样得到的结果就有好坏，非常不确定，所以运行n_init，取其中最好的作为初始的中心点。如果K值比较大的时候，你可以适当增大n_init这个值。
- init：即初始值选择的方式，默认是采用的是优化过的k-means++方式，你也可以自己指定中心点，或者采用random的完全随机的方式，一般推荐采用优化过的k-means++的方式；
- algorithm：k-means的实现算法，有"auto" ,"full", "elkan"三种。一般来说建议直接使用默认的“auto”。
    - 三者区别：如果选择"full"采用的是传统的K-Means算法，"auto"会根据数据的特点自动选择是选择“full”，还是“elkan”。所以一般默认选择“auto”。


在创建好K-Means类之后，就可以使用它的方法，最常用的是fit和predict这两个函数。我们可以单独使用fit函数和predict函数，也可以合并使用fit_predict函数。其中fit(data)可以对data数据进行K-Means聚类。predict(data)可以针对data中的每一个样本，计算最近的类。

下面我们可以完整的跑一遍20支亚洲球队的聚类问题：

In [4]:
#coding:utf-8
from sklearn.cluster import KMeans
from sklearn import preprocessing
import pandas as pd
import numpy as np
# 输入数据
data = pd.read_csv('data.csv',encoding='gbk')
# 进行特征选择
train_x = data[["2019年国际排名","2018世界杯","2015亚洲杯"]]
df = pd.DataFrame(train_x)
# 随机取到K的值为3
kmeans = KMeans(n_clusters=3)
# 规范化到[0,1]空间
min_max_scaler = preprocessing.MinMaxScaler()
train_x=min_max_scaler.fit_transform(train_x)
# kmeans 算法
kmeans.fit(train_x)
predict_y = kmeans.predict(train_x)
# 合并聚类结果，插入到原数据中
result = pd.concat((data,pd.DataFrame(predict_y)),axis=1)
result.rename({0:u'聚类'}, axis=1, inplace=True)
print(result)


        国家  2019年国际排名  2018世界杯  2015亚洲杯  聚类
0       中国         73       40        7   2
1       日本         60       15        5   1
2       韩国         61       19        2   1
3       伊朗         34       18        6   1
4       沙特         67       26       10   1
5      伊拉克         91       40        4   2
6      卡塔尔        101       40       13   0
7      阿联酋         81       40        6   2
8   乌兹别克斯坦         88       40        8   2
9       泰国        122       40       17   0
10      越南        102       50       17   0
11      阿曼         87       50       12   0
12      巴林        116       50       11   0
13      朝鲜        110       50       14   0
14      印尼        164       50       17   0
15      澳洲         40       30        1   1
16     叙利亚         76       40       17   0
17      约旦        118       50        9   0
18     科威特        160       50       15   0
19    巴勒斯坦         96       50       16   0


## 总结

1、开始的三个问题的回答：

i)如何确定K类的中心点？
- 初始的时候需要设置参数n_init的值，然后选择初始中心点效果最好的设置为初始值。在每次更新之后，我们都需要重新确认每一类中心点，一般采用均值的方式进行确认。

ii)如何将其他的点划分到K类中？
- 可以根据距离的定义：欧氏距离、曼哈顿距离、切比雪夫距离、余弦距离等中的一种进行计算，对于点的划分，就看他距离那个类的中心点的距离最近，就属于哪一类。

iii)如何区分K-Means和KNN这两种算法？
- 可以从三个维度来进行区分
- 1、首先，这两个算法解决数据挖掘的两类为题。K-Means是聚类算法，KNN是分类算法。
- 2、这两个算法分别是两种不同的学习方式。K-Means是非监督学习，也就是不需要事先给出分类的标签，而KNN是用监督的学习，需要我们给出训练数据的分类标识。
- 3、最后，K值的含义不同。K—Means中的K值代表K类。KNN中的K值代表K个最接近的邻居

2、K-Means:
- 工作原理
    - step1：选取K个点作为初始的中心点。
    - Step2：将每个点分配到最近的类中心点，然后重新计算每个类的中心点。
    - Step3: 重复Step2，直到类不发生变化，或者达到最大的迭代次数为止。
- sklearn
    - sklearn_cluster提供了9种聚类的方法
    - 引用：from sklearn.cluster import KMeans
    - 创建KMeans()
        - n_cluster:K值
        - max_iter：最大的迭代次数
        - n_init：初始化中心点的运算次数
        - init：初始值选择的方式，默认是k-means++的方式
        - algorithm：kmeans的实现算法，默认是auto
    - 功能调用
        - fit(data):对data数据进行K-Means聚类
        - predict(data):针对data中的每个样本，计算最近的类
- K—Means和KNN的区别
    - 分类or聚类：K—Means是聚类，KNN是分类
    - 监督学习or非监督学习：K-Means非监督学习，KNN监督学习
    - K值含义：K-Means中代表K类，KNN中代表K个最接近的邻居
    

3、课后练习：将上面的20支亚洲球队使用K-Means划分成为5类，在规范化数据的时候采用标准化的处理方式（即均值为0，方差为1），如何编写程序？

In [1]:
#coding:utf-8
from sklearn.cluster import KMeans
from sklearn import preprocessing
import pandas as pd
import numpy as np
# 输入数据
data = pd.read_csv('data.csv',encoding='gbk')
# 进行特征选择
train_x = data[["2019年国际排名","2018世界杯","2015亚洲杯"]]
df = pd.DataFrame(train_x)
# 随机取到K的值为3
kmeans = KMeans(n_clusters=5)
# 规范化，标准zheng正态分布，均值为0，方差为1：
min_max_scaler = preprocessing.StandardScaler()
train_x=min_max_scaler.fit_transform(train_x)
# kmeans 算法进行拟合
kmeans.fit(train_x)
# 对结果进行预测
predict_y = kmeans.predict(train_x)
# 合并聚类结果，插入到原数据中
result = pd.concat((data,pd.DataFrame(predict_y)),axis=1)
result.rename({0:u'聚类'}, axis=1, inplace=True)
print(result)


        国家  2019年国际排名  2018世界杯  2015亚洲杯  聚类
0       中国         73       40        7   0
1       日本         60       15        5   2
2       韩国         61       19        2   2
3       伊朗         34       18        6   2
4       沙特         67       26       10   2
5      伊拉克         91       40        4   0
6      卡塔尔        101       40       13   4
7      阿联酋         81       40        6   0
8   乌兹别克斯坦         88       40        8   0
9       泰国        122       40       17   1
10      越南        102       50       17   1
11      阿曼         87       50       12   4
12      巴林        116       50       11   4
13      朝鲜        110       50       14   4
14      印尼        164       50       17   3
15      澳洲         40       30        1   2
16     叙利亚         76       40       17   1
17      约旦        118       50        9   4
18     科威特        160       50       15   3
19    巴勒斯坦         96       50       16   1
